In [195]:
n = 3
indexes = [str(i) for i in range(1, n+1)]

def one_index_names(name):
	return [name +"_"+ i  for i in indexes]

def two_index_names(name):
	return [name +"_"+ i + j for i in indexes for j in indexes if i != j]

def jjpi_names(name):
    return [name +"_"+ j + jp + "_" + i for j in indexes for jp in indexes for i in indexes if (j !=jp and j != i and jp != i)]

def v(name, i, j = None, k=None):
    #print name, i, j,k
    if name == "f" or name == "h":
        ring = Rring
    else:
        ring = Uring
    if j is None:
        return ring(name + "_" + i)
    if k is None:
        return ring(name + "_" + i + j)
    return Uring(name + "_" + i + j  + "_" +k)

In [196]:
def h2_rel(i):
    return (v("h",i)^2, v("f",i)^3 + v("pi",i)*v("f",i) + v("s",i))
    
def fh_rel(i,j):
    return (v("f",i)*v("h",i,j), v("a",i,j)*v("h",i) + v("b",i,j)*v("h",i,j) + v("a",i,j)*v("e",i,j))

def hh_rel(i,j):
    return (v("h",i)*v("h",i,j), v("a",i,j)*v("f",i)^2 + v("e",i,j)*v("h",i,j) + v("a",i,j)*v("b",i,j)*v("f",i) + v("a",i,j)*(v("pi",i) + v("b",i,j)^2))

In [277]:
Uring = PolynomialRing(QQ, one_index_names("pi") + one_index_names("s") + two_index_names("a") + two_index_names("b") 
                      +two_index_names("e") + jjpi_names("c") + one_index_names("ga") + one_index_names("del") +
                       one_index_names("ep") + one_index_names("theta") + two_index_names("ac") + two_index_names("alin")
                      + two_index_names("asq")+ one_index_names("xx") + one_index_names("yy") +  two_index_names("blin") +
                       two_index_names("elin") + two_index_names("bsq") + two_index_names("esq") + ["aaa","bbb","ccc","ddd"])

Rring = PolynomialRing(Uring, one_index_names("f") + one_index_names("h") + two_index_names("h"))

In [278]:
ExpandRing.<t> = LaurentSeriesRing(Uring)

expand_at_dict = dict()

for j in indexes: #expand at j
    expand_at_dict[j] = dict()
    for i in indexes: #looking at f_i, h_i
        f = v("f",i)
        h = v("h",i)
        if i != j:
            expand_at_dict[j][f] = v("b",i,j)  + v("blin",i,j)*t +  v("bsq",i,j)*t^2 + O(t^3)
            expand_at_dict[j][h] = v("e",i,j)  + v("elin",i,j)*t +  v("esq",i,j)*t^2 + O(t^3)
        else:
            expand_at_dict[j][f] = 1/t^2  + v("del",j)*t + v("pi",j)*t^2 + O(t^3)
            #ga = 0;  v("ga",j)
            expand_at_dict[j][h] = 1/t^3 + 3/2*v("del",j) + v("theta",j)*t + O(t^2)
            #-3*del_1 + 2*ep_1


        
        for k in indexes:
            if i == k:
                continue
            hS = v("h",i,k)
            
            if j == i:
                expand_at_dict[j][hS] = v("a",i,k)/t  + v("a",i,k)*v("b",i,k)*t + v("a",i,k)*( v("e",i,k) - 1/2*v("del",i) )*t^2 + O(t^3)
                #ac = 0; + v("ac",i,k)
                #-a_12*b_12 + alin_12 == 0
                #-a_12*e_12 + a_12*del_1 - a_12*ep_1 + asq_12
                #asq_12 = -(-a_12*e_12 + a_12*del_1 - a_12*ep_1)
                #asq_12 = a_12(e_12 -del_1 + ep_1)
                #-3*del_1 + 2*ep_1;; ep_1 = 3/2del_1
                #aqs_12 = a_12(e_12 -1/2del_1)
                
            elif j == k:
                expand_at_dict[j][hS] = 1/t + v("xx",j) + v("yy",j)*t + O(t^2)
            else:
                expand_at_dict[j][hS] = v("c", k, j, i) + O(t)


                
def expand_at(func, point):
    return func.subs(expand_at_dict[point])


def find_rels_by_expansion(point_index, eqn):
    expansion = expand_at(eqn[0]-eqn[1], point_index)
    
    print "Some relations"
    for value in expansion.coefficients():
        print value


In [279]:
find_rels_by_expansion("1",h2_rel("1"))
find_rels_by_expansion("1",h2_rel("2"))

find_rels_by_expansion("1", fh_rel("1","2"))
find_rels_by_expansion("1", fh_rel("2","1"))

find_rels_by_expansion("1", hh_rel("1","2"))
find_rels_by_expansion("1", hh_rel("2","1"))


Some relations
-4*pi_1 + 2*theta_1
Some relations
-b_21^3 - pi_2*b_21 + e_21^2 - s_2
-3*b_21^2*blin_21 - pi_2*blin_21 + 2*e_21*elin_21
-3*b_21*blin_21^2 - 3*b_21^2*bsq_21 + elin_21^2 - pi_2*bsq_21 + 2*e_21*esq_21
Some relations
-a_12*del_1
Some relations
-2*a_21*e_21 + blin_21
xx_1*blin_21 - a_21*elin_21 + bsq_21
Some relations
-a_12*del_1
Some relations
-3*a_21*b_21^2 - pi_2*a_21 + elin_21
-3*a_21*b_21*blin_21 + xx_1*elin_21 + esq_21


In [280]:
def transition(H0gens, f, coefs_to_check):
    """
    This function will find the coefficients of the expression of f in the basis given H0gens. 
    coefs_to_check is a list of pairs (point, exponent) of terms in the expansion to use to 
    determine the coefficients.
    
    Note: The user is responsible for making sure that the parameters are compatible!!
    """
    H0gens = [Rring(g) for g in H0gens]
    f = Rring(f)
    
    M = Matrix(Uring, len(coefs_to_check),len(H0gens),)
    b = vector(Uring,len(coefs_to_check))

    for i, (point, expon) in enumerate(coefs_to_check):
        #print expand_at(f, point), expon
        b[i] = expand_at(f,point)[expon]
        for j,gen in enumerate(H0gens):
            #print expand_at(gen,point), expon
            M[i,j] = expand_at(gen,point)[expon]
    print
    for row in M.rows():
        print row
    print
    print b
    print
            
    result = M.augment(b).echelon_form()
    for row in result.rows():
        print row
        
    return result

In [281]:
transition(["h_21^2", "h_21" ,"f_2","1"],
          "f_1",
          [("1",-2),("2",-2),("2",-1),("2",0)])
f_1_2 = Rring("h_21^2 - a_21^2 * f_2  -2*a_21^2*b_21 + b_12")


(1, 0, 0, 0)
(a_21^2, 0, 1, 0)
(0, a_21, 0, 0)
(2*a_21^2*b_21, 0, 0, 1)

(1, 0, 0, b_12)

(1, 0, 0, 0, 1)
(0, 0, 1, 0, -a_21^2)
(0, 0, 0, 1, -2*a_21^2*b_21 + b_12)
(0, a_21, 0, 0, 0)


In [282]:
transition(["h_21^3", "h_21^2", "h_21","h_2","f_2","1"],
          "h_1",
          [("1",-3),("2",-3),("2",-2),("2",-1),("2",0)])
h_1_2 = Rring("h_21^3 + aaa*h_21^2-3*a_21^2*b_21*h_21 - a_21^3*h_2 + bbb*f_2 + ccc")


(1, 0, 0, 0, 0, 0)
(a_21^3, 0, 0, 1, 0, 0)
(0, a_21^2, 0, 0, 1, 0)
(3*a_21^3*b_21, 0, a_21, 0, 0, 0)
(3*a_21^3*e_21 - 3/2*a_21^3*del_2, 2*a_21^2*b_21, 0, 3/2*del_2, 0, 1)

(1, 0, 0, 0, e_12)

(1, 0, 0, 0, 0, 0, 1)
(0, 0, 0, 1, 0, 0, -a_21^3)
(0, a_21^2, 0, 0, 1, 0, 0)
(0, 2*a_21^2*b_21, 0, 0, 0, 1, -3*a_21^3*e_21 + 3*a_21^3*del_2 + e_12)
(0, 0, a_21, 0, 0, 0, -3*a_21^3*b_21)


In [283]:
expand_at(h_1_2^2 - f_1_2^3,"1")

2*aaa*t^-5 + (3*a_21^2*b_21 + 10*xx_1*aaa + aaa^2 - 3*b_12)*t^-4 + (-2*a_21^3*e_21 + 12*a_21^2*b_21*xx_1 - 6*a_21^2*b_21*aaa + 3*a_21^2*blin_21 + 20*xx_1^2*aaa + 4*xx_1*aaa^2 - 12*b_12*xx_1 + 10*yy_1*aaa + 2*b_21*bbb + 2*ccc)*t^-3 + O(t^-2)

In [284]:
transition(["h_21^3", "h_21","h_2","f_2","1"],
          "h_1",
          [("1",-3),("2",-3),("2",-2),("2",-1),("2",0)])
h_1_2 = Rring("h_21^3 - 3*a_21^2*b_21*h_21 - a_21^3*h_2  + -3*a_21^3*e_21 + 3*a_21^3*del_2 + e_12")


(1, 0, 0, 0, 0)
(a_21^3, 0, 1, 0, 0)
(0, 0, 0, 1, 0)
(3*a_21^3*b_21, a_21, 0, 0, 0)
(3*a_21^3*e_21 - 3/2*a_21^3*del_2, 0, 3/2*del_2, 0, 1)

(1, 0, 0, 0, e_12)

(1, 0, 0, 0, 0, 1)
(0, 0, 1, 0, 0, -a_21^3)
(0, 0, 0, 1, 0, 0)
(0, 0, 0, 0, 1, -3*a_21^3*e_21 + 3*a_21^3*del_2 + e_12)
(0, a_21, 0, 0, 0, -3*a_21^3*b_21)


In [289]:
expand_at(h_1_2^2 - f_1_2^3,"1")

(3*a_21^2*b_21 - 3*b_12)*t^-4 + (-8*a_21^3*e_21 + 6*a_21^3*del_2 + 12*a_21^2*b_21*xx_1 + 3*a_21^2*blin_21 - 12*b_12*xx_1 + 2*e_12)*t^-3 + (-18*a_21^4*b_21^2 - 24*a_21^3*e_21*xx_1 + 18*a_21^3*del_2*xx_1 + 18*a_21^2*b_21*xx_1^2 + 18*a_21^2*b_12*b_21 + 12*a_21^2*b_21*yy_1 + 12*a_21^2*xx_1*blin_21 - 2*a_21^3*elin_21 - 18*b_12*xx_1^2 + 3*a_21^2*bsq_21 - 3*b_12^2 + 6*e_12*xx_1 - 12*b_12*yy_1)*t^-2 + O(t^-1)

In [286]:
h_1_2^2 - f_1_2^3

3*a_21^2*f_2*h_21^4 + (-3*a_21^4)*f_2^2*h_21^2 + (-2*a_21^3)*h_2*h_21^3 + (-3*b_12)*h_21^4 + a_21^6*f_2^3 + (-12*a_21^4*b_21 + 6*a_21^2*b_12)*f_2*h_21^2 + (-6*a_21^3*e_21 + 6*a_21^3*del_2 + 2*e_12)*h_21^3 + (6*a_21^6*b_21 - 3*a_21^4*b_12)*f_2^2 + a_21^6*h_2^2 + 6*a_21^5*b_21*h_2*h_21 + (-3*a_21^4*b_21^2 + 12*a_21^2*b_12*b_21 - 3*b_12^2)*h_21^2 + (12*a_21^6*b_21^2 - 12*a_21^4*b_12*b_21 + 3*a_21^2*b_12^2)*f_2 + (6*a_21^6*e_21 - 6*a_21^6*del_2 - 2*a_21^3*e_12)*h_2 + (18*a_21^5*b_21*e_21 - 18*a_21^5*b_21*del_2 - 6*a_21^2*b_21*e_12)*h_21 + 8*a_21^6*b_21^3 + 9*a_21^6*e_21^2 - 18*a_21^6*e_21*del_2 + 9*a_21^6*del_2^2 - 12*a_21^4*b_12*b_21^2 + 6*a_21^2*b_12^2*b_21 - 6*a_21^3*e_12*e_21 + 6*a_21^3*e_12*del_2 - b_12^3 + e_12^2

In [287]:
expand_at(Rring("(-3*a_21^4)*f_2^2*h_21^2 + (-2*a_21^3)*h_2*h_21^3 + (-3*b_12)*h_21^4 + a_21^6*f_2^3 + (-12*a_21^4*b_21 + 6*a_21^2*b_12)*f_2*h_21^2 + 2*ccc*h_21^3 + (6*a_21^6*b_21 - 3*a_21^4*b_12)*f_2^2 + a_21^6*h_2^2 + 6*a_21^5*b_21*h_2*h_21 + (-3*a_21^4*b_21^2 + 12*a_21^2*b_12*b_21 - 3*b_12^2)*h_21^2 + (12*a_21^6*b_21^2 - 12*a_21^4*b_12*b_21 + 3*a_21^2*b_12^2)*f_2 + (-2*a_21^3*ccc)*h_2 + (-6*a_21^2*b_21*ccc)*h_21 + 8*a_21^6*b_21^3 - 12*a_21^4*b_12*b_21^2 + 6*a_21^2*b_12^2*b_21 - b_12^3 + ccc^2"),"1")


-3*b_12*t^-4 + (-2*a_21^3*e_21 - 12*b_12*xx_1 + 2*ccc)*t^-3 + (-18*a_21^4*b_21^2 - 6*a_21^3*e_21*xx_1 + 18*a_21^2*b_12*b_21 - 2*a_21^3*elin_21 - 18*b_12*xx_1^2 - 3*b_12^2 - 12*b_12*yy_1 + 6*xx_1*ccc)*t^-2 + O(t^-1)

Hence aaa = 0 and we get b_12 = a_12^2*b_21 as desired

In [288]:
lms = [ (Rring("h_1^2"), h2_rel, ("1",)),
        (Rring("h_2^2"), h2_rel, ("2",)),
        (Rring("f_1*h_12"), fh_rel, ("1","2")),
        (Rring("f_2*h_21"), fh_rel, ("2","1")),
        (Rring("h_1*h_12"), hh_rel, ("1","2")),
        (Rring("h_2*h_21"), hh_rel, ("2","1"))]

def doS(r1,r2):
    """
    Implements the Buchberger algorithm to find relations between the coefficients.
    """
    L = lcm(r1[0].change_ring(QQ), r2[0].change_ring(QQ))
    factor1 = L.quo_rem(r1[0])[0]
    factor2 = L.quo_rem(r2[0])[0]
    S = factor1 * r1[1] - factor2 * r2[1]
    newS = reduce_lm(S)

    if newS == 0:
        print "No relations found"
    else:
        for c in newS.coefficients():
            print c
        
def reduce_lm(fff):
    """
    Outsource this code from doS.
    """
    fff1 = fff
    done = False
    
    while not done:
        new_fff = 0
        for m in fff1.monomials():
            can_be_reduced = False
            for lm in lms:
                if lm[0].divides(m):
                    can_be_reduced = True
                    #newS += make_rel(lm[1], lm[2], lm[3])[1] *(m/lm[0]).numerator()*S1.monomial_coefficient(m)
                    new_fff += (lm[1](*lm[2]))[1] * (m/lm[0]).numerator()*fff1.monomial_coefficient(m)
                    break
            if not can_be_reduced:
                new_fff += m*fff1.monomial_coefficient(m)


        if fff1 == new_fff:
            done = True
        else:
            #print "need to go again"
            fff1 = new_fff
    return new_fff

In [273]:
doS(h2_rel("1"),fh_rel("1","2"))
doS(h2_rel("2"),fh_rel("1","2"))  
doS(fh_rel("1","2"), fh_rel("2","1"))
doS(hh_rel("1","2"), hh_rel("2","1"))
doS(hh_rel("1","2"), fh_rel("2","1"))
doS(hh_rel("1","2"), fh_rel("1","2"))
doS(hh_rel("1","2"), h2_rel("2"))
doS(hh_rel("1","2"), h2_rel("1"))

No relations found
No relations found
No relations found
No relations found
No relations found
-a_12*b_12^3 - pi_1*a_12*b_12 + a_12*e_12^2 - s_1*a_12
No relations found
-b_12^3 - pi_1*b_12 + e_12^2 - s_1
